<a href="https://colab.research.google.com/github/dKC-7000001/RAG_resume/blob/main/RAG_resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ResuRAG

In [1]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.9 MB/s eta 0:00:00


In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 18.6 MB/s eta 0:00:00


In [3]:
## Embedding
!pip install install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.6 MB/s eta 0:00:00


In [11]:
!pip install llama_index

In [14]:
!pip install llama-index-llms-huggingface

In [15]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [18]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

[Document(id_='202c973c-0558-47bc-812f-aaa0dac57e37', embedding=None, metadata={'page_label': '1', 'file_name': 'Abhishek_Resume_MLE_Gurugram.pdf', 'file_path': '/content/data/Abhishek_Resume_MLE_Gurugram.pdf', 'file_type': 'application/pdf', 'file_size': 59212, 'creation_date': '2024-04-14', 'last_modified_date': '2024-04-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Summary\nMachine Learning Engineer with over 2 years of hands-on experience in modeling, anomaly detection, data\nvisualization, statistical analysis, ﬁne-tuning using Python. Proﬁcient in problem-solving and resolving code issues.\nStrong foundation in Engineering and Mathematics, specializing in Industrial Engineering and Management.\nSkills\nMachine Learning, Artiﬁcial Intelligen

In [19]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [21]:
query_wrapper_prompt

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='<|USER|>{query_str}<|ASSISTANT|>')

In [20]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [8]:
!pip show llama_index

Name: llama-index
Version: 0.10.28
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse
Required-by: 


In [24]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [31]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
# from llama_index.core.embeddings import LangchainEmbedding
from llama_index.legacy.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-32-f6dd2f399444>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [33]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ee0a0aa2a40>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ee0a0aa2a40>, id_func=<function default_id_func at 0x7ee186343be0>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7ee0a0aa1090>, callback_manager=<llama_index.core.callbacks.ba

In [34]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [35]:
index

In [36]:
query_engine=index.as_query_engine()

In [41]:
response=query_engine.query("What are the technologies abhishek knows?")

In [42]:
print(response)

Abhishek has knowledge of the following technologies:

1. AWS Textract
2. Selenium
3. Requests
4. Open-source libraries including BeautifulSoup, pytesseract, easyocr, paddleOCR, tika, tabula, and pdfplumber
5. Python
6. NLP
7. Power BI
8. MSSQL
9. TensorFlow
10. LoRA

Note: The answer is based on the information provided in the context and may not be exhaustive or up-to-date.


In [45]:
response=query_engine.query("In which company is Abhishek currently working?")

In [44]:
print(response)

Abhishek works in Mercedes-Benz Research and Development as a summer intern.


In [46]:
response=query_engine.query("What is Abhishek current profile?")

In [47]:
print(response)

Abhishek's current profile is a Machine Learning Engineer at Applied Solar Technologies India Private Limited in Gurugram, India. He has over 2 years of experience in modeling, anomaly detection, data visualization, statistical analysis, and fine-tuning using Python. Abhishek has a strong foundation in Engineering and Mathematics, specializing in Industrial Engineering and Management. His skills include Machine Learning, Artificial Intelligence, Deep Learning, Transformers, Fine Tunning, Computer Vision, Statistics, Regression, Classification, Clustering, Forecasting, Sentiment Analysis, Outlier Detection, Python, Data Visualization, NLP, TensorFlow, Keras, Pandas, Numpy, Matplotlib, Seaborn, Plotly, Sklearn, Docker, AWS Lambda, AWS Textract, AWS ECR, AWS Event bridge, Git, MS SQL, Power BI, Flask, RPA, Selenium, MS Excel, Redis, BeautifulSoup, and more.


In [48]:
response=query_engine.query("Give me some education background?")

In [49]:
print(response)

Of course! Based on the context information provided, Abhishek has a Bachelor of Technology (B.Tech) degree in Mechanical Engineering from KIET in Ghaziabad, Uttar Pradesh, India, which he completed in 2017 with a percentage of 69%. He then went on to pursue a Master of Technology (M.Tech) degree in Industrial Engineering and Management from National Institute of Technology (NIT) Trichy in Tiruchirappalli, Tamil Nadu, India, which he completed in 2022 with an average percentage of 8.83 on a scale of 10.


In [51]:
response=query_engine.query("What are the personnel projects that he have done?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [52]:
print(response)

Abhishek has worked on several personal projects, including:

1. Fine Tunning of Gemma using LoRA to make the text response more understandable and clear.
2. Predictive Maintenance of Machine/Equipments using Machine Learning - M.Tech Final year Project.
3. Forecasting of outsourced parts for BHEL Trichy with LSTM using TensorFlow.
4. Credit card fraud detection using Logistic regression.
5. NIT Trichy Sentiment Analysis.
6. Divorce Detection using Neural Network.

Note: The answer is based on the information provided in the context and may not be exhaustive or up-to-date.


In [53]:
response=query_engine.query("As an AI, can you recommend me that, if this candidate is good for job profile which involes machine learning job which requires 2 years of experience?")

In [54]:
print(response)


Based on the information provided in the resume, Abhishek Chaudhary has over 2 years of experience in machine learning, specifically in modeling, anomaly detection, data visualization, and statistical analysis using Python. He has a strong foundation in engineering and mathematics, with a specialization in industrial engineering and management.
Given this information, it is likely that Abhishek Chaudhary would be a good fit for a job profile that involves machine learning. His experience and skills in machine learning, coupled with his strong foundation in engineering and mathematics, make him a promising candidate for a role that requires proficiency in these areas.
However, it is important to note that the suitability of a candidate for a particular job role also depends on other factors such as their communication skills, teamwork abilities, and adaptability to new technologies and workflows. Therefore, it is essential to conduct a thorough interview and assessment process to deter

In [55]:
response=query_engine.query("As an AI, can you recommend me that as there is no time for interview and assesment, if this candidate is good for job profile which involes machine learning job which requires 2 years of experience?")

In [56]:
print(response)


Based on the information provided in the resume, the candidate, Abhishek Chaudhary, has over 2 years of experience in machine learning, specifically in modeling, anomaly detection, data visualization, and statistical analysis. He has also worked on various projects involving machine learning, including battery state of health forecasting, tariff drift detection, and outlier detection.
Given the lack of time for an interview and assessment, it is possible to consider Abhishek for a machine learning job without conducting a thorough evaluation process. However, it is essential to conduct a brief initial screening to verify the candidate's qualifications and ensure they meet the minimum requirements for the position.
Some questions that could be asked during this initial screening include:

1. Can you tell me about your experience in machine learning?
2. What are some of the projects you have worked on in the field of machine learning?
3. How do you stay up-to-date with the latest develo